<a href="https://colab.research.google.com/github/dearxcorex/ASR_MODEL_NBTC/blob/main/ASR_modal/text_classification/dev_test/data/torchtext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### NLP from scratch with Pytorch

[Pytorch](https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html)



In [ ]:
!pip install pythainlp deepcut

In [43]:
from pythainlp import word_tokenize
import pandas as pd


# Pytorch
from torch.utils.data import random_split, DataLoader
#from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [3]:
df = pd.read_csv("clean_text.csv", encoding="utf-8")

In [4]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [5]:
df.head()

,tweet_text,is_toxic
0,วันๆ นี่คุยกะหมา แมว หมู ไก่ ม้า ควาย มากกว่าค...,0
1,หล่อมากกก หล่อวัวตายควายล้มกันเลยทีเดียวววว,0
2,สิวเหี้ยไรขึ้นหลังหู เสียชาติเกิดมั้ย เกิดมามั...,0
3,อ่ะ ป่วยก็ป่วย งานจ้างก็ต้องทำ งานเรียนก็ต้องท...,0
4,นี่ก็เพิ่งรู้ว่า เกิดชาตินี้ชาติเดียว เป็นทั้ง...,1


In [36]:
df['tweet_text'] = df['tweet_text'].apply(lambda x: word_tokenize(x, engine="newmm"))

In [37]:
vocab = build_vocab_from_iterator(df['tweet_text'], specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [39]:
vocab(['นาย', 'นาง', 'นางสาว'])

[600, 599, 0]

In [44]:
train_df, test_df = random_split(df, [0.8, 0.2])

In [45]:
train_dataloader = DataLoader(train_df, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_df, batch_size=32, shuffle=True)